In [1]:
from sampler import sample_from_logreg
import numpy as np
import matplotlib.pyplot as plt
from og_gd import run_sim
import time

In [2]:
def pi(theta):
    return np.linalg.norm(theta, 2)


def F(X, y, theta, lbd=1e-6):
    return np.sum(l(X, y, theta)) + lbd * pi(theta)


def nabla_F(X, y, theta=None, lbd=1e-6):
    if y.shape[0] == 1 or y.shape[0] == 0:
        return (
            -(X.T * y) + X.T * ((np.exp(X @ theta)) / (1 + np.exp(X @ theta))) + lbd * theta
        )
        
    return (
        -(X.T @ y) + X.T @ ((np.exp(X @ theta)) / (1 + np.exp(X @ theta))) + lbd * theta
    )

def hess_F(X, theta=None, lbd=1e-6):
    expy = np.exp(X @ theta) / (1 + np.exp(X @ theta)) ** 2
    if len(X.shape) < 2:
        X = X.reshape(-1, 1)
        return X @ X.T * expy + lbd
    return X.T * expy @ X + lbd * np.eye(theta.shape[0])

In [3]:
start = time.time()
res = run_sim(250, 20, n_iter=250)
print(np.mean(res["IACV"]))
print(np.mean(res["NS"]))
print(np.mean(res["IJ"]))
end = time.time()
print(end - start)

0.0002772612244955063
0.003732731115163448
0.00607059717145864
7.662547826766968


In [6]:
start = time.time()
p = 20
n = 250
n_iter = 250
X, _, y = sample_from_logreg(p=p, n=n)

lbd_v = 1e-6 * n

theta = np.zeros(p)
alpha = 0.5 / n
alpha_t = alpha

theta_cv = np.zeros((n, p))
theta_true = np.zeros((n, p))
theta_ns = np.zeros((n, p))
theta_ij = np.zeros((n, p))

err_approx = {
    "IACV": np.zeros(n_iter),
    "NS": np.zeros(n_iter),
    "IJ": np.zeros(n_iter),
    "hat": np.zeros(n_iter),
}

for t in range(0, n_iter):
    f_grad = nabla_F(X, y, theta, lbd=lbd_v)
    f_hess = hess_F(X, theta, lbd=lbd_v)
    
    hess_per_sample = np.vectorize(hess_F, excluded={"theta", "lbd"}, signature='(p)->(p, p)')(X, theta=theta, lbd=lbd_v)
    grad_per_sample = np.vectorize(nabla_F, excluded={"theta", "lbd"}, signature='(p),(0)->(p)')(X, y.reshape(-1, 1), theta=theta, lbd=lbd_v)
    
    hess_minus_i = f_hess - hess_per_sample
    grad_minus_i = f_grad - grad_per_sample

    theta_cv = theta_cv - alpha_t * grad_minus_i - alpha_t * np.vectorize(np.matmul, signature='(p, p),(p)->(p)')(hess_minus_i, (theta_cv - theta))
    
    theta_ns = theta + np.vectorize(np.matmul, signature='(p, p),(p)->(p)')(np.linalg.inv(hess_minus_i), grad_per_sample)
    theta_ij = theta + np.vectorize(np.matmul, signature='(p, p),(p)->(p)')(np.linalg.inv(f_hess), grad_per_sample)

    for i in range(0, n):
        theta_true[i] = theta_true[i] - alpha * nabla_F(
            np.delete(X, (i), axis=0),
            np.delete(y, (i), axis=0),
            theta=theta_true[i],
            lbd=lbd_v,
        )

    # actually update theta
    theta = theta - alpha * f_grad

    err_approx["IACV"][t] = np.mean(np.linalg.norm(theta_cv - theta_true, 2, axis=1))
    err_approx["NS"][t] = np.mean(np.linalg.norm(theta_ns - theta_true, 2, axis=1))
    err_approx["IJ"][t] = np.mean(np.linalg.norm(theta_ij - theta_true, 2, axis=1))
    err_approx["hat"][t] = np.mean(np.linalg.norm(theta - theta_true, 2, axis=1))


print(np.mean(err_approx["IACV"]))
print(np.mean(err_approx["NS"]))
print(np.mean(err_approx["IJ"]))
end = time.time()
print(end - start)

0.00027726070745688055
0.00373273162848995
0.00607059717145864
4.340312719345093
